# QCEngine

QCEngine is a quantum chemistry abstraction layer where many different quantum chemistry (or quantum chemistry like!) programs can be run with identical input and ouput abstractions that match the MolSSI QCSchema.

In [2]:
import qcelemental as qcel
import qcengine as qcng

In [3]:
qcng.list_all_programs()

{'dftd3',
 'entos',
 'molpro',
 'mopac',
 'mp2d',
 'psi4',
 'rdkit',
 'terachem',
 'torchani'}

In [5]:
qcng.list_available_programs()

{'dftd3', 'mopac', 'psi4', 'rdkit'}

In [8]:
mol = qcel.models.Molecule(geometry=[[0, 0, 0], [0, 1.5, 0], [0, 0, 1.5]],
                           symbols=["O", "H", "H"],
                           connectivity=[[0, 1, 1], [0, 2, 1]])
mol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

<Molecule(name='H2O' formula='H2O' hash='b41d0c5')>

In [13]:
qctask = {
    "molecule": mol,
    "driver": "energy",
    "model": {"method": "B3LYP", "basis": "6-31g"}
}
ret = qcng.compute(qctask, "psi4")

In [15]:
ret.return_result

-76.27412972063468

In [18]:
ret.provenance.dict()

{'creator': 'Psi4',
 'version': '1.3.2',
 'routine': 'psi4.json.run_json',
 'qcengine_version': 'v0.9.0',
 'hostname': 'Daniels-MacBook-Pro.local',
 'memory': 2.531,
 'nthreads': 2,
 'wall_time': 1.06595778465271,
 'cpu': 'Intel(R) Core(TM) i7-7820HQ CPU @ 2.90GHz',
 'username': 'daniel'}

In [19]:
ret.properties.dict()

{'calcinfo_nbasis': 13,
 'calcinfo_nmo': 13,
 'calcinfo_nalpha': 5,
 'calcinfo_nbeta': 5,
 'calcinfo_natom': 3,
 'nuclear_repulsion_energy': 11.138071187457696,
 'return_energy': -76.27412972063468,
 'scf_one_electron_energy': -126.25159666378755,
 'scf_two_electron_energy': 46.55689518291617,
 'scf_vv10_energy': None,
 'scf_xc_energy': -7.717499427220995,
 'scf_dispersion_correction_energy': None,
 'scf_dipole_moment': [0.0, 0.0, 2.6607950246342504],
 'scf_total_energy': -76.27412972063468,
 'scf_iterations': 6,
 'mp2_same_spin_correlation_energy': None,
 'mp2_opposite_spin_correlation_energy': None,
 'mp2_singles_energy': None,
 'mp2_doubles_energy': None,
 'mp2_total_correlation_energy': None,
 'mp2_correlation_energy': None,
 'mp2_total_energy': None,
 'mp2_dipole_moment': None,
 'ccsd_same_spin_correlation_energy': None,
 'ccsd_opposite_spin_correlation_energy': None,
 'ccsd_singles_energy': None,
 'ccsd_doubles_energy': None,
 'ccsd_correlation_energy': None,
 'ccsd_total_energy'